# Chainerによるセマンティックセグメンテーション

##本チュートリアルではchainerを利用してセマンティックセグメンテーションの実装を確認，学習および評価を行います．　環境としてはGoogle が提供する Google Colaboratory上でおこないます． GPU上で処理を行うため，colaboratoryの[ランタイム]->[ランタイムのタイプを変更]からハードウェアアクセラレータをGPUにしてください．

Goolge Colaboratory上にChainerとChainerCVをインストールします．

In [ ]:
!curl https://colab.chainer.org/install | sh -
!pip install 'chainercv'

!pip install tqdm

学習したモデルをgoogle driveに保存するための処理を行います．途中でgoogle driveにアクセスするためのキー入力が求められます．その際，表示されたURLをクリックし，アクセスの許可をするとキーが表示されます．そのキーをコピーし，キー入力のエリアに貼り付けしてください．キー入力は２回求められます．それぞれ別々のキーなので，それぞれのURLをクリックして，同様の手順で行ってください．

まずはSegNetの推論処理を実行します．セグメンテーションに必要なモジュールや関数をインポートします．

In [ ]:
from collections import defaultdict
import numpy as np
import matplotlib.pyplot as plt

import chainer

from chainercv.datasets import camvid_label_colors
from chainercv.datasets import camvid_label_names
from chainercv.datasets import CamVidDataset
from chainercv.links import SegNetBasic
from chainercv import utils
from chainercv.visualizations import vis_image
from chainercv.visualizations import vis_semantic_segmentation


ここではCamVidというデータセットで学習されたSegNetBasicをモデルとして利用します．

In [ ]:
chainer.config.train = False
premodel_name ="camvid"
model = SegNetBasic(n_class=len(camvid_label_names),pretrained_model=premodel_name)
model.to_gpu()

テストに用いる画像ファイル名を指定します．
左側の矢印メニューをクリックし，ファイルを選択します．
アップロードより，検出処理させたい画像をアップロードします．
そのファイル名をimage_nameに代入します．
img_nameの画像を読み込み，セグメンテーション処理を行います．

In [ ]:
img_name="28554775_10216169965424085_897703047_o.jpg"
img = utils.read_image(img_name, color=True)
labels = model.predict([img])
label = labels[0]

元画像とセグメンテーション結果画像を表示します． chainerCVでは，vis_semantic_segmentationという画像を表示するためのクラスが用意されています．これは，セグメンテーション処理結果をもとにクラスごとに色分けした画像を作成してくれます．

In [ ]:
def clearLabel(_ax):
  _ax.tick_params(labelbottom="off",bottom="off")
  _ax.tick_params(labelleft="off",left="off")
  _ax.set_xticklabels([]) 
  _ax.axis('off')
  return _ax

fig = plt.figure()
ax1 = fig.add_subplot(1, 2, 1)
clearLabel(ax1)
vis_image(img, ax=ax1)
ax2 = fig.add_subplot(1, 2, 2)
clearLabel(ax2)
vis_semantic_segmentation( None, label, camvid_label_names, camvid_label_colors, ax=ax2)
plt.show()

次に，セグメンテーションの学習を行います．学習に必要なパッケージ類をインポートします



In [ ]:
import chainer
import numpy as np
from functools import partial
import time


from chainer.datasets import TransformDataset
from chainer import optimizers, cuda, Variable

from chainercv.datasets import camvid_label_names
from chainercv.datasets import CamVidDataset
from chainercv.links import PixelwiseSoftmaxClassifier
from chainercv.links import SegNetBasic

chainer.config.train = True

In [ ]:
!cp -r "drive/Colab Notebooks/DL_Tutorial/camvid" ./

In [ ]:
!ls

data augmentationを行う関数transformを用意します．ここでは，画像の左右反転をランダムに行うようにします．

In [ ]:
def transform(in_data):
    img, label = in_data
    if np.random.rand() > 0.5:
        img = img[:, :, ::-1]
        label = label[:, ::-1]
    return img, label

学習と評価にはCamVidというデータセットを用います．　chainerCVではCamVidのデータセットをダウンロードし，学習と評価のデータセットに分けることができます．

In [ ]:
train_dataset = CamVidDataset(data_dir="camvid", split='train')
val_dataset    = CamVidDataset(data_dir="camvid", split='val')

学習データにdata augmentationを施しながらデータを取得するように高階関数partialを用いて，TransformDatasetを用意します．

In [ ]:
train_transform = partial(transform)
valid_transform = partial(transform)

train_dataset = TransformDataset(train_dataset, train_transform)
val_dataset    = TransformDataset(val_dataset, valid_transform)

CamVidデータセットには木や歩行者，道路など11クラスあり，各クラスの出現確率はバラバラです．そのため，学習時に出現確率が高いクラスに偏った学習がされてしまいます．そこで，クラスの出現確率を考慮した重みを用意します．これは，出現確率が低いほど大きな重みとし，その重みを誤差逆伝播時に利用します．これにより，出現確率の低いクラスの誤差をより大きく更新に反映させることができます．

In [ ]:
n_class = 11
dataset = CamVidDataset(data_dir="camvid",split='train')

n_cls_pixels = np.zeros((n_class,))
n_img_pixels = np.zeros((n_class,))

for img, label in dataset:
    for cls_i in np.unique(label):
        if cls_i == -1:
            continue
        n_cls_pixels[cls_i] += np.sum(label == cls_i)
        n_img_pixels[cls_i] += label.size
freq = n_cls_pixels / n_img_pixels
median_freq = np.median(freq)
class_weight = median_freq / freq

ネットワーク構造を定義します．SegNetの構造はSegNetBasicとして用意されています．また，セマンティックセグメンテーションの場合は各画素に対して識別を行うので，PixelwiseSoftmaxClassifierを利用します．

In [ ]:
xp = cuda.cupy
model = SegNetBasic(n_class=len(camvid_label_names))
model = PixelwiseSoftmaxClassifier( model, class_weight = class_weight)
model.to_gpu()

最適化手法にはモーメンタム付きのSGDを利用します．

In [ ]:
optimizer = optimizers.MomentumSGD(lr=0.1, momentum=0.9)
optimizer.setup(model)
optimizer.add_hook(chainer.optimizer_hooks.WeightDecay(rate=0.0005))

学習を行います．

In [ ]:
batch_size = 12
epoch_num = 100
    
start = time.time()
for epoch in range(epoch_num):
    dataset_x = []
    dataset_label =[]
    train_data_num = len(train_dataset)
    iter_one_epoch = int(train_data_num/batch_size)
    sum_loss = 0
#    if (epoch+1) % 25 == 0 :
#        optimizer.lr *= 0.1
    perm = np.random.permutation(train_data_num)
    for i in range(len(train_dataset)):
        train_sample = train_dataset.get_example(perm[i])
        dataset_x.append(train_sample[0])
        dataset_label.append(train_sample[1])

        if len(dataset_x)==batch_size:
            train_x = xp.asarray(dataset_x, xp.float32)
            train_label = xp.asarray(dataset_label, xp.int32)
            dataset_x = []
            dataset_label =[]         

            x = Variable(cuda.to_gpu(train_x))
            glabel = Variable(cuda.to_gpu(train_label))
            loss = model(x, glabel)        
            model.zerograds()
            loss.backward()
            optimizer.update()
            sum_loss += loss.data
    elapsed_time = time.time() - start            
    print("epoch: {}, mean loss: {}, proc. time {}".format(epoch+1, sum_loss*batch_size/train_data_num, elapsed_time))

    if (epoch+1) %5 == 0:
        chainer.serializers.save_hdf5('drive/Colab Notebooks/DL_Tutorial/SemSeg_Camvid_{}.npz'.format(epoch+1), model, compression=6)


In [ ]:
!ls "drive/Colab Notebooks/DL_Tutorial/"

In [ ]:
!ls "drive/Colab Notebooks/DL_Tutorial/SemSeg_Camvid_100.npz"

In [ ]:
chainer.config.train = False
premodel_name ="camvid"
test_model = SegNetBasic(n_class=len(camvid_label_names))
test_model = PixelwiseSoftmaxClassifier( test_model, class_weight = class_weight)
chainer.serializers.load_hdf5("drive/Colab Notebooks/DL_Tutorial/SemSeg_Camvid_5.npz", test_model)
test_model.to_gpu()

img_name="drive/Colab Notebooks/DL_Tutorial/0006R0_f02040.png"
img = utils.read_image(img_name, color=True)
x = Variable(cuda.to_gpu(xp.asarray(img, xp.float32) ))

y = test_model.predictor([x])


In [ ]:
import numpy as np
import os
import time

import chainer
from chainercv.links import SegNetBasic
from chainercv.datasets import CamVidDataset
from chainercv.datasets import camvid_label_names
from chainercv.links import PixelwiseSoftmaxClassifier

from chainercv import utils
from chainer import optimizers, cuda, Variable

In [ ]:

data_dir="drive/Colab Notebooks/DL_Tutorial/camvid"
split="train"
img_list_path = os.path.join(data_dir, '{}.txt'.format(split))
train_dataset = [  [os.path.join(data_dir, fn.replace('/SegNet/CamVid/', ''))   for fn in line.split()] for line in open(img_list_path)]


In [ ]:
def transform(in_data):
    img, label = in_data
    if np.random.rand() > 0.5:
        img = img[:, :, ::-1]
        label = label[:, ::-1]
    return img, label

In [ ]:
n_class = 11
dataset = CamVidDataset(data_dir="drive/Colab Notebooks/DL_Tutorial/camvid",split='train')

n_cls_pixels = np.zeros((n_class,))
n_img_pixels = np.zeros((n_class,))

for img, label in dataset:
    for cls_i in np.unique(label):
        if cls_i == -1:
            continue
        n_cls_pixels[cls_i] += np.sum(label == cls_i)
        n_img_pixels[cls_i] += label.size
freq = n_cls_pixels / n_img_pixels
median_freq = np.median(freq)
class_weight = median_freq / freq

In [ ]:
xp = cuda.cupy
model = SegNetBasic(n_class=len(camvid_label_names))
model = PixelwiseSoftmaxClassifier( model, class_weight = class_weight)
model.to_gpu()

In [ ]:
optimizer = optimizers.MomentumSGD(lr=0.1, momentum=0.9)
optimizer.setup(model)
optimizer.add_hook(chainer.optimizer_hooks.WeightDecay(rate=0.0005))

In [ ]:
from chainercv.utils import read_image

batch_size = 12
epoch_num = 100
    
for epoch in range(epoch_num):
    start = time.time()
    sum_loss = 0
    train_data_num = len(train_dataset)
    iter_one_epoch = int(train_data_num/batch_size)
    dataset_x = []
    dataset_label =[]
    perm = np.random.permutation(train_data_num)

    for i in range(len(train_dataset)):
        data_path , label_path = train_dataset[perm[i]]
        img = read_image(data_path, color=True)
        label = read_image(label_path, dtype=np.int32, color=False)[0]
        label[label == 11] = -1
        img, label = transform([img, label])

        dataset_x.append(img)
        dataset_label.append(label)

        if (len(dataset_x) == batch_size):
            train_x = xp.asarray(dataset_x, xp.float32)
            train_label = xp.asarray(dataset_label, xp.int32)
            dataset_x = []
            dataset_label =[]         
            
            x = Variable(cuda.to_gpu(train_x))
            glabel = Variable(cuda.to_gpu(train_label))
            loss = model(x, glabel)        
            model.zerograds()
            loss.backward()
            optimizer.update()
            sum_loss += loss.data
    elapsed_time = time.time() - start            
    print("epoch: {}, mean loss: {}, proc. time {}".format(epoch+1, sum_loss*batch_size/train_data_num, elapsed_time))


    


